# Исследование уровня NPS у клиентов телекоммуникационной компании

Заказчик исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.

Цель - определить текущий уровень потребительской лояльности (NPS) среди клиентов из России и оформить результаты в виде дашборда Tableau.

### Подключение к базе

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'

In [3]:
query = """
SELECT * FROM location;
"""

In [4]:
location = pd.read_sql(query, engine)
location.head(3)

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия


In [5]:
location.to_csv('location.csv', index=False)

In [6]:
location = pd.read_csv('location.csv')

Проверили, что данные выгружаются правильно. Приступаем к выгрузке нужных данных для анализа.

### Выгрузка данных

In [7]:
query = '''
        SELECT
        u.user_id,
        u.lt_day,
        CASE
            WHEN u.lt_day < 365 THEN 'новый'
            ELSE 'опытный'
        END AS 'is_new',
        u.age,
        CASE 
            WHEN u.gender_segment = 1 THEN 'женщина'
            WHEN u.gender_segment = 0 THEN 'мужчина'
        END AS 'gender_segment',
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        SUBSTR(ag.title, 3) as age_segment,
        SUBSTR(tr.title, 3) as traffic_segment,
        SUBSTR(lf.title, 3) as lifetime_segment,
        u.nps_score,
        CASE 
            WHEN u.nps_score <= 6 THEN 'критик'
            WHEN u.nps_score >= 9 THEN 'сторонник'
            ELSE 'нейтрал'
        END AS nps_group
        FROM user AS u
        JOIN location AS l ON u.location_id = l.location_id
        JOIN age_segment AS ag ON u.age_gr_id = ag.age_gr_id
        JOIN traffic_segment AS tr ON u.tr_gr_id = tr.tr_gr_id
        JOIN lifetime_segment AS lf ON u.lt_gr_id = lf.lt_gr_id
        WHERE age NOT NULL
        
        '''

In [8]:
data = pd.read_sql(query, engine) 

In [9]:
data.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,опытный,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонник
1,A001WF,2344,опытный,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонник
2,A003Q7,467,опытный,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонник
3,A004TB,4190,опытный,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонник
4,A004XT,1163,опытный,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонник


In [10]:
data.to_csv('data.csv', index=False)

In [11]:
data = pd.read_csv('data.csv')

### Ссылка на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/ksenia.antonova/viz/Telecomm_NPS/Dashboard1

### Ссылка на файл с презентацией:

https://docs.google.com/presentation/d/1XmiRD6JKuA1PMU0sPC51mProz1qPcxNx38fp8GrbK5c/edit#slide=id.p1